In [1]:
import os
import sys
import ccxt
import pandas as pd
from pathlib import Path

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.utils import get_absolute_path, get_config, read_file

In [4]:
exchange_id = 'binance'

In [5]:
try:
    exchange = getattr(ccxt, exchange_id)()
except AttributeError:
    print(f"Error: The exchange '{exchange_id}' is not supported by ccxt.")

In [6]:
start_date = '2024-01-01'
end_date = '2025-01-01'

In [7]:
since = exchange.parse8601(f'{start_date}T00:00:00Z')
end_timestamp = exchange.parse8601(f'{end_date}T23:59:59Z')

In [8]:
symbol = 'ETH/USDT'
timeframe = '1d'

In [9]:
all_ohlcv = []
while since < end_timestamp:
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)

            if not ohlcv:
                break

            all_ohlcv.extend(ohlcv)

            since = ohlcv[-1][0] + 1

        except ccxt.NetworkError as e:
            print(f"Network error: {e}. Retrying...")
            continue
        except ccxt.ExchangeError as e:
            print(f"Exchange error: {e}")

In [10]:
df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

In [11]:
df

,timestamp,open,high,low,close,volume
0,1704067200000,2281.87,2352.37,2265.24,2352.04,2.167027e+05
1,1704153600000,2352.05,2431.30,2341.00,2355.34,4.580417e+05
2,1704240000000,2355.35,2385.45,2100.00,2209.72,7.987295e+05
3,1704326400000,2209.72,2294.69,2201.91,2267.11,4.488441e+05
4,1704412800000,2267.11,2277.21,2206.17,2268.78,4.055680e+05
...,...,...,...,...,...,...
495,1746835200000,2345.04,2600.00,2317.00,2583.23,9.793502e+05
496,1746921600000,2583.22,2608.13,2435.51,2514.57,8.315844e+05
497,1747008000000,2514.58,2624.00,2406.63,2495.47,1.068681e+06
498,1747094400000,2495.48,2738.50,2415.30,2679.71,1.031356e+06


In [12]:
config = get_config.read_yaml()
data_dir = get_absolute_path.absolute(config['paths']['data_directory'])
print(config['paths']['data_directory'])
data_dir

data


PosixPath('/Users/aayushsharma/Projects/Project-catRL/data')

In [13]:
data_path = Path(data_dir)
data_path.mkdir(parents=True, exist_ok=True)

In [14]:
symbol = config['data']['tickers']

In [15]:
symbol

['ETH/USDT', 'BTC/USDT']

In [16]:
df.to_csv(data_dir / 'OHLCV.csv', index=False)

In [17]:
data = read_file.read_data('OHLCV')

Reading Raw Equity data: OHLCV !
Successfully read: OHLCV !


In [18]:
data

,timestamp,open,high,low,close,volume
0,1704067200000,2281.87,2352.37,2265.24,2352.04,2.167027e+05
1,1704153600000,2352.05,2431.30,2341.00,2355.34,4.580417e+05
2,1704240000000,2355.35,2385.45,2100.00,2209.72,7.987295e+05
3,1704326400000,2209.72,2294.69,2201.91,2267.11,4.488441e+05
4,1704412800000,2267.11,2277.21,2206.17,2268.78,4.055680e+05
...,...,...,...,...,...,...
495,1746835200000,2345.04,2600.00,2317.00,2583.23,9.793502e+05
496,1746921600000,2583.22,2608.13,2435.51,2514.57,8.315844e+05
497,1747008000000,2514.58,2624.00,2406.63,2495.47,1.068681e+06
498,1747094400000,2495.48,2738.50,2415.30,2679.71,1.031356e+06
